# Limpieza de DF de Proyecto

Importamos las librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
import re

Leemos el .csv y mostramos todas las columnas

In [ ]:
rh  = pd.read_csv("datos_limpios.csv", index_col = 0)
pd.set_option("display.max.columns", None)
#pd.set_option("display.max.rows", None)

Estandarizamos los nombres de columnas: todo en minúsculas y espacios como _


In [ ]:
# Primer paso: las columnas de todo en mayuscula, pasamos a minúsculas y separamos las palabras#

rh.rename(columns={'NUMCOMPANIESWORKED':'num_companies_worked', 'TOTALWORKINGYEARS':'total_working_years', 'WORKLIFEBALANCE':'work_life_balance','YEARSWITHCURRMANAGER':'years_with_curr_manager','NUMBERCHILDREN':'number_children'},inplace = True)

In [ ]:
# Segundo paso:todo en minuscula, añadir barra baja separadora

# Patrón regex para identificar la transición de minúsculas a mayúsculas
patron = '(?<=[a-z])(?=[A-Z])'

# Diccionario para almacenar los nuevos nombres de columna
columnas = {column: re.sub(patron,'_', column) for column in rh.columns}

# Renombrar las columnas usando los nuevos nombres
rh.rename(columns=columnas, inplace=True)

# Convertir los nombres de las columnas a minúsculas
rh.columns = [columna.lower() for columna in rh.columns]

#cambiamos manualmente el nombre de una columna
rh.rename(columns = {'employeenumber': 'employee_number', 'employeecount' : 'employee_count'}, inplace = True )

Estandarizamos booleanos : general: Y/N, genre: F/M

In [ ]:
# Para género
rh["gender"].replace(to_replace = [0,1], value = ['M', 'F'], inplace = True)

In [ ]:
#Para Y/N
rh["over18"].replace(to_replace = [0,1], value = ['N', 'Y'], inplace = True)
rh[["attrition", "over_time"]]= rh[["attrition", "over_time"]].replace(to_replace = ["No","Yes"], value = ['N', 'Y']) #Borrar

In [ ]:

rh["remote_work"].replace(to_replace = ["0","1"], value = ['N', 'Y'], inplace = True)
rh["remote_work"].replace(to_replace = ["No","Yes"], value = ['N', 'Y'], inplace = True)
rh["remote_work"].replace(to_replace = ["False","True"], value = ['N', 'Y'], inplace = True)

Eliminamos las columnas no necesarias.

In [ ]:
# .drop standard_hours, number_children, salary, over18, employee_count
rh.drop(labels=["standard_hours", "number_children", "salary", "over18", "employee_count", "years_in_current_role", "same_as_monthly_income", "date_birth"], axis = 1, inplace=True)

Estandarizamos todos los valores en camel

In [ ]:
def camelizar (dato):

    try:
        return dato.title()
    except:
        return dato
    
columnas_object_camel = ["age","attrition","business_travel","daily_rate","department","education_field","employee_number","hourly_rate","job_role","marital_status","monthly_income","over_time","performance_rating","total_working_years","work_life_balance","role_departament","remote_work"]

for columnas in columnas_object_camel:
    rh[columnas]=rh[columnas].apply(camelizar)


Cambiamos comas por puntos en WorkLifeBalance, performace_rating y monthly_income quitar comas

In [ ]:
columnas_sin_comas = ['work_life_balance','performance_rating','monthly_income']
for columna in columnas_sin_comas:
    rh[columna] = rh[columna].str.replace(",",".")


De las columnas de salario quitamos $ y pasamos a tipo a int

In [ ]:
columnas_sin_dollar = ['daily_rate']
for columna in columnas_sin_dollar:
    rh[columna] = rh[columna].str.replace('$','').str.replace(",",".")

rh[columna]
rh["daily_rate"]= rh["daily_rate"].astype(float)

Sustituimos el Not Available por nan en hourly_rate y convertimos a float:

In [ ]:
# hourly_date
rh['hourly_rate']=rh['hourly_rate'].replace('Not Available', np.nan)
rh['hourly_rate']=rh['hourly_rate'].astype('float', errors='ignore')

Pasamos a float las columnas de monthly_income, performance_rating y work_life_balance (las demás object). daily_rate(float64) 

In [ ]:
def pasar_a_float(elemento):
    try:
        return float(elemento)
    except (ValueError, TypeError):
        return np.NaN

lista_columnas = ['monthly_income', 'performance_rating', 'work_life_balance'] 

for columna in lista_columnas:
    rh[columna] = rh[columna].apply(pasar_a_float).astype('float')

# replace de not available por numpyNaN #especificar el texto que sale 
# gestion de nulos
# conversión

Comprobamos los porcentajes de nulos

In [ ]:
info_columnas = pd.concat([(rh.isnull().sum() / rh.shape[0]) * 100, rh.dtypes], axis=1)
info_columnas.columns = ['Porcentaje de Nulos','Tipos de Columnas']
info_columnas

Corregimos errores tipográficos en algunos valores de las columnas categóricas. Por ejemplo, en la columna MaritalStatus en vez de "Married" en algunas filas aparece "Marreid".

In [ ]:
rh['marital_status']= rh['marital_status'].replace({'Marreid':'Married'})


Eliminamos valores negativos de distance_from_home

In [ ]:
rh["distance_from_home"] = rh["distance_from_home"].where(rh["distance_from_home"] >= 0, np.nan)

#No admite NaN en columnas int, tienen que ser float, por lo que ha añadido .0 y no se puede cambiar, exige entonces cambiar los NaN 


Cambiamos a float la columna total_working_years

In [ ]:
rh['total_working_years'] = rh['total_working_years'].str.replace(",", ".")

rh['total_working_years'] = rh['total_working_years'].astype(float)


Cambiamos los nulos de marital_status por unknown

In [ ]:
rh['marital_status'] = rh['marital_status'].replace(np.nan,'Unknown', regex = True)

Cambiamos los nulos de worklife_balance y performace_rating por la moda

In [ ]:
media = rh['work_life_balance'].mean()
moda = rh['work_life_balance'].mode()[0]  
print(f'La media es {media}')
print(f'La moda es {moda}')

In [ ]:
rh['work_life_balance'] = rh['work_life_balance'].fillna(moda)

rh['work_life_balance'].isnull().sum()

Quitamos el segundo dígito de environment_satisfaction

In [ ]:
rh['environment_satisfaction'] = rh['environment_satisfaction'].apply(lambda x: x // 10 if x > 9 else x)

Borramos columnas con una cantidad de nulos que supera nuestro estandar

In [ ]:
columnas = ['business_travel', 'employee_number' , 'monthly_income', 'over_time', 'department', 'education_field', 'role_departament', 'daily_rate', 'hourly_rate']

rh.drop(columnas, axis = 1, inplace = True)

Cambiamos las cifras de age que están en letra por su número correspondiente

In [ ]:
rh['age'] = rh['age'].replace("Forty-Seven", "47")
rh['age'] = rh['age'].replace("Fifty-Eight", "58")
rh['age'] = rh['age'].replace("Thirty-Six", "36")
rh['age'] = rh['age'].replace("Fifty-Five", "55")
rh['age'] = rh['age'].replace("Fifty-Two", "52")
rh['age'] = rh['age'].replace("Thirty-One", "31")
rh['age'] = rh['age'].replace("Thirty", "30")
rh['age'] = rh['age'].replace("Twenty-Six", "26")
rh['age'] = rh['age'].replace("Thirty-Seven", "37")
rh['age'] = rh['age'].replace("Thirty-Two", "32")
rh['age'] = rh['age'].replace("Twenty-Four", "24")

Sustituimos los nulos de Hourly_rate sustituir por el valor de sus vecinos usando una nueva columna y eliminamos la antigua

In [ ]:
from sklearn.impute import KNNImputer
imputer_knn = KNNImputer(n_neighbors = 5)
hourly_rate_knn = imputer_knn.fit_transform(rh[["hourly_rate"]])
hourly_rate_knn
rh[["hourly_rate_i"]] = hourly_rate_knn
rh[["hourly_rate_i"]].isnull().sum()

In [ ]:
columnas = ['hourly_rate']

rh.drop(columnas, axis = 1, inplace = True)

In [ ]:
rh.rename(columns = {'hourly_rate_i': 'hourly_rate'}, inplace = True)

Comprobamos los porcentajes de nulos

In [12]:
info_columnas = pd.concat([(rh.isnull().sum() / rh.shape[0]) * 100, rh.dtypes], axis=1)
info_columnas.columns = ['Porcentaje de Nulos','Tipos de Columnas']
info_columnas

,Porcentaje de Nulos,Tipos de Columnas
age,0.000000,object
attrition,0.000000,object
business_travel,47.831475,object
daily_rate,7.682776,float64
department,81.288724,object
distance_from_home,0.000000,int64
education,0.000000,int64
education_field,46.158612,object
employee_number,26.703841,object
environment_satisfaction,0.000000,int64


Guardamos un nuevo .csv con los datos limpios

In [ ]:
rh.to_csv('datos_limpios_final.csv')